# Daily Planner Agent  
### *A Multi-Agent Productivity System *

This project demonstrates how AI agents can work together to improve daily productivity by extracting tasks, prioritizing them, scheduling them intelligently, and evaluating the final plan.

---

## Goals
- Convert scatared user input into clean tasks  
- Prioritize tasks based on importance  
- Schedule them efficiently into the user's day  
- Store user preferences using memory  
- Evaluate the final schedule automatically  

---

## Key AI Agent Concepts Implemented
- **Multi-agent architecture (4 agents)**
- **Tool usage (Python execution as a scheduling tool)**
- **Custom Tool (calendar slot generator)**
- **Memory / Session state**
- **Agent evaluation loop**
- **Context management**


# System Architecture


---

## Agents Used
1. **TaskExtractorAgent** → Extracts clean tasks from messy notes  
2. **PriorityEvaluatorAgent** → Scores tasks based on importance  
3. **SchedulerAgent** → Places tasks into timeline slots  
4. **ScheduleQualityAgent** → Evaluates final schedule  

---

## Tools Used
- **Python code execution** for scheduling & time calculations  
- **Custom calendar tool** to generate day slots  

---

## Memory Used
Stored preferences:
- Wake time  
- Sleep time  
- Preferred task duration  
- Break duration  

Used throughout scheduling.


In [1]:
import re
from datetime import datetime, timedelta

USER_MEMORY = {
    "wake_time": "09:00",
    "sleep_time": "23:00",
    "default_task_duration": 45,  # minutes per task
    "break_duration": 10          # minutes break
}
print("Memory initialized.")


Memory initialized.


# Agent Definitions

Below i have define four agent:

### 1. **TaskExtractorAgent**
Extracts task from collection of text.

### 2. **PriorityEvaluatorAgent**
Assigns scores to tasks and sorts them.

### 3. **SchedulerAgent**
Schedules tasks into daily time slots using a custom tool.

### 4. **ScheduleQualityAgent**
Evaluates the quality of the final plan.


In [2]:
def TaskExtractorAgent(user_input: str):
    """
    Extracts tasks from messy text input.
    Simulates LLM extraction using simple rule-based filtering.
    """
    
    lines = [x.strip("-• ").strip() for x in user_input.split("\n") if len(x.strip()) > 2]
    
    tasks = []
    for line in lines:
        line = re.sub(r'[^\w\s:]', '', line)
        tasks.append(line)
    
    return tasks

print("TaskExtractorAgent ready.")


TaskExtractorAgent ready.


In [3]:
def PriorityEvaluatorAgent(tasks):
    """
    Assigns priority scores using keyword detection.
    Simulates an evaluator LLM agent.
    """

    priority_map = {
        "urgent": 10,
        "important": 8,
        "call": 7,
        "meeting": 6,
        "email": 5,
        "study": 7,
        "submit": 9,
    }
    evaluated = []
    for task in tasks:
        score = 3 
        for keyword, value in priority_map.items():
            if keyword in task.lower():
                score = max(score, value)
        evaluated.append((task, score))
    evaluated.sort(key=lambda x: x[1], reverse=True)
    return evaluated

print("PriorityEvaluatorAgent ready.")


PriorityEvaluatorAgent ready.


In [4]:
def get_daily_slots():
    """
    Generates usable slots between wake_time and sleep_time.
    Each slot = task_duration + break.
    """

    slots = []
    start = datetime.strptime(USER_MEMORY["wake_time"], "%H:%M")
    end = datetime.strptime(USER_MEMORY["sleep_time"], "%H:%M")
    step = timedelta(minutes=USER_MEMORY["default_task_duration"] + USER_MEMORY["break_duration"])

    current = start
    while current + timedelta(minutes=USER_MEMORY["default_task_duration"]) <= end:
        slots.append(current.strftime("%H:%M"))
        current += step
    return slots

print("Calendar slot tool ready.")


Calendar slot tool ready.


In [5]:
def SchedulerAgent(evaluated_tasks):
    """
    Schedules tasks into time slots.
    Uses the custom calendar tool + memory.
    """
    slots = get_daily_slots()
    schedule = []

    for i, (task, score) in enumerate(evaluated_tasks):
        if i < len(slots):
            schedule.append({
                "time": slots[i],
                "task": task,
                "priority": score
            })
    
    return schedule

print("SchedulerAgent ready.")


SchedulerAgent ready.


# ScheduleQualityAgent (Evaluation)

This agent will evaluates the final schedule to find out the:

- Balance  
- Overload  
- High-priority tasks placed first  
- No time conflicts  
- Proper spacing  

A score from **1 to 10** is returned.


In [6]:
def ScheduleQualityAgent(schedule):
    """
    Simple scoring model evaluating:
    - High-priority tasks earlier
    - Number of tasks
    - Spacing
    """
    score = 5  
    
    if not schedule:
        return 1, "No tasks scheduled."
    for i in range(len(schedule)-1):
        if schedule[i]["priority"] < schedule[i+1]["priority"]:
            score -= 1  
        else:
            score += 1  
    if USER_MEMORY["break_duration"] >= 10:
        score += 1
    score = max(1, min(score, 10))
    
    return score, "Evaluation complete."

print("ScheduleQualityAgent ready.")


ScheduleQualityAgent ready.


#  Run the Multi-Agent Pipeline

Enter your notes or tasks in the cell below.


In [7]:
user_input = """
- urgent: submit assignment
- call the bank
- study DSA
- email professor
- important: resume update
"""

print("USER INPUT:\n", user_input)
tasks = TaskExtractorAgent(user_input)
print("\nEXTRACTED TASKS:\n", tasks)
evaluated = PriorityEvaluatorAgent(tasks)
print("\nPRIORITIZED TASKS:\n", evaluated)
schedule = SchedulerAgent(evaluated)
print("\nFINAL SCHEDULE:\n", schedule)
score, msg = ScheduleQualityAgent(schedule)
print("\nSCHEDULE SCORE:", score)
print("COMMENT:", msg)


USER INPUT:
 
- urgent: submit assignment
- call the bank
- study DSA
- email professor
- important: resume update


EXTRACTED TASKS:
 ['urgent: submit assignment', 'call the bank', 'study DSA', 'email professor', 'important: resume update']

PRIORITIZED TASKS:
 [('urgent: submit assignment', 10), ('important: resume update', 8), ('call the bank', 7), ('study DSA', 7), ('email professor', 5)]

FINAL SCHEDULE:
 [{'time': '09:00', 'task': 'urgent: submit assignment', 'priority': 10}, {'time': '09:55', 'task': 'important: resume update', 'priority': 8}, {'time': '10:50', 'task': 'call the bank', 'priority': 7}, {'time': '11:45', 'task': 'study DSA', 'priority': 7}, {'time': '12:40', 'task': 'email professor', 'priority': 5}]

SCHEDULE SCORE: 10
COMMENT: Evaluation complete.


# Conclusion

I have successfully built a **multi-agent productivity system** that demonstrates:

- Task extraction  
- Priority scoring  
- Intelligent scheduling using tools  
- Memory-driven preferences  
- Evaluation of output quality  

* points covered
* 
✔ Multi-agent system  
✔ Tools (execution + custom tool)  
✔ Session memory  
✔ Evaluation agent  
✔ Sequential pipeline  

i can scale this into project can be extended into:
- Email integration  
- Calendar APIs  
- Natural-language LLM extraction  
- Auto reminders  
